In [1]:
from pathlib import Path

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa

from rasterio.enums import Resampling

In [2]:
DATA_DIR = Path('/data/nga/')
out_dir = Path('/data/nga/ncs')

tifs = list(DATA_DIR.glob('*SLC/*georeferenced.tif'))

meta = pd.read_excel(Path('/home/rdcrlzh1s/radarsat/data/RS2_collection.xlsx'), parse_dates=['Acq. Date'])
dss = {name:None for name in meta['AOI '].unique()}


In [3]:
meta

,AOI,Acq. Date,Start time,Orbit,Abs orb,Beam,Polarity,inc. angle,lat/long center,lat/long NW,lat/long NE,lat/long SW,lat/long SE
0,Utqiagvik,2023-06-01,17:07:18.540000,234-223D,80721.30,SLA11,HH,38.08,71°19'N/156°45'W,71°22'N/156°59'W,71°20'N/156°28',71°18'N/157°01'W,71°16'N/156°31'
1,Oliktok,2023-06-02,03:01:19.280000,234-229A,80727.20,SLA76,HH,26.12,70°30'N/149°56'W,70°30'N/150°12'W,70°33'N/149°43',70°27'N/150°09'W,70°30'N/149°40'
2,Toolik,2023-06-02,16:38:28.010000,234-237D,80735.31,U5W2,HH,32.38,69°04'N/148°44'W,69°21'N/149°10'W,69°14'N/147°59',68°55'N/149°28'W,68°48'N/148°18'
3,Toolik,2023-06-15,03:21:43.170000,235-72A,80913.19,U11W2,HH,37.42,69°06'N/148°51'W,69°17'N/149°29'W,69°22'N/148°28',68°51'N/149°13'W,68°56'N/148°13'
4,Utqiagvik,2023-06-15,16:58:57.190000,235-80D,80921.30,SLA16,HH,41.71,71°19'N/156°38'W,71°22'N/156°52'W,71°20'N/156°21',71°18'N/156°54'W,71°15'N/156°23'
5,Pituffik,2023-06-15,21:52:11.470000,235-83A,80924.21,U18,HH,43.08,76°32'N/68°38'W,76°36'N/69°05'W,76°38'N/68°22'W,76°26'N/68°55'W,76°28'N/68°12'W
6,Oliktok,2023-06-16,16:29:48.100000,235-94D,80935.30,SLA16,HH,41.71,70°29'N/149°47'W,70°33'N/150°00'W,70°31'N/149°31',70°28'N/150°03'W,70°26'N/149°33'
7,Pituffik,2023-06-22,21:48:00.520000,235-183A,81024.21,U16,HH,41.71,76°32'N/68°41'W,76°36'N/69°07'W,76°38'N/68°25'W,76°25'N/68°56'W,76°28'N/68°14'W
8,Toolik,2023-07-09,03:21:43.170000,236-72A,81256.19,U11W2,HH,37.42,69°06'N/148°51'W,69°17'N/149°29'W,69°22'N/148°28',68°51'N/149°13'W,68°56'N/148°13'
9,Utqiagvik,2023-07-09,16:58:57.190000,236-80D,81264.30,SLA16,HH,41.71,71°19'N/156°38'W,71°22'N/156°52'W,71°20'N/156°21',71°18'N/156°54'W,71°15'N/156°23'


In [55]:
DATA_DIR = Path('/data/nga/')
out_dir = Path('/data/nga/ncs')

tifs = list(DATA_DIR.glob('*SLC/*georeferenced.tif'))

meta = pd.read_excel(Path('/home/rdcrlzh1s/radarsat/data/RS2_collection.xlsx'), parse_dates=['Acq. Date'])
dss = {name:None for name in meta['AOI '].unique()}

for tif in tifs[7:9]:
    platform, order_key, product_key, delivery_key, beam_mode, date, time, pol, processing_level = tif.parent.stem.split('_')
    image_meta = meta.loc[(meta['Beam'] == beam_mode) & (meta['Acq. Date'] == date)]
    print(image_meta['AOI '])
    print(image_meta['Acq. Date'])

    da = xr.open_dataarray(tif).isel(x = slice(0, -1, 1000), y = slice(0, -1, 1000))

    result = 1j*da.isel(band = 1); result += da.isel(band = 0)
    mag = 10*np.log10(np.abs(result)).drop('band').rename('magntiude')
    mag = mag.where(np.isfinite(mag))

    phase = xr.apply_ufunc(np.angle, result).drop('band').rename('phase')

    date_ds = xr.merge([result.rename('data'), mag, phase])

    date_ds = date_ds.expand_dims(time = [image_meta['Acq. Date'].iloc[0]])

    print(dss[image_meta['AOI '].iloc[0]])

    if type(dss[image_meta['AOI '].iloc[0]]) != xr.Dataset:
        print('starting')
        dss[image_meta['AOI '].iloc[0]] = date_ds
    
    else:
        print('appending')
        date_ds = date_ds.rio.reproject_match(dss[image_meta['AOI '].iloc[0]], nodata = np.nan, resampling=Resampling.bilinear)
        dss[image_meta['AOI '].iloc[0]] = xr.concat([dss[image_meta['AOI '].iloc[0]], date_ds], 'time')
    
    # f"{image_meta['AOI '].iloc[0]}_{image_meta['Polarity'].iloc[0]}_{image_meta['Acq. Date'].dt.strftime('%Y-%m-%d').iloc[0]}.nc"

14    Toolik
Name: AOI , dtype: object
14   2023-07-20
Name: Acq. Date, dtype: datetime64[ns]


/home/rdcrlzh1s/miniforge3/envs/science/lib/python3.12/site-packages/xarray/core/computation.py:821: RuntimeWarning: divide by zero encountered in log10
  result_data = func(*input_data)


None
starting
15    Toolik
Name: AOI , dtype: object
15   2023-08-02
Name: Acq. Date, dtype: datetime64[ns]
<xarray.Dataset>
Dimensions:      (time: 1, x: 49, y: 18)
Coordinates:
  * time         (time) datetime64[ns] 2023-07-20
    band         int64 2
  * x            (x) float64 -149.5 -149.5 -149.5 ... -148.0 -148.0 -148.0
  * y            (y) float64 69.36 69.33 69.3 69.27 ... 68.91 68.88 68.85 68.81
    spatial_ref  int64 0
Data variables:
    data         (time, y, x) complex64 0j 0j 0j 0j 0j 0j ... 0j 0j 0j 0j 0j 0j
    magntiude    (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
    phase        (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
appending


/home/rdcrlzh1s/miniforge3/envs/science/lib/python3.12/site-packages/xarray/core/computation.py:821: RuntimeWarning: divide by zero encountered in log10
  result_data = func(*input_data)


In [56]:
for k, v in dss.items():
    v.to_netcdf(out_dir.joinpath(f'{k}.nc'))

{'Utqiagvik': None,
 'Oliktok': None,
 'Toolik': <xarray.Dataset>
 Dimensions:      (time: 2, x: 49, y: 18)
 Coordinates:
   * time         (time) datetime64[ns] 2023-07-20 2023-08-02
     band         int64 2
   * x            (x) float64 -149.5 -149.5 -149.5 ... -148.0 -148.0 -148.0
   * y            (y) float64 69.36 69.33 69.3 69.27 ... 68.91 68.88 68.85 68.81
     spatial_ref  int64 0
 Data variables:
     data         (time, y, x) complex64 0j 0j 0j ... (nan+0j) (nan+0j) (nan+0j)
     magntiude    (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan
     phase        (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan,
 'Pituffik': None}